# Task 1a

## cross validation for ridge regression

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import numdifftools as nd
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, Ridge

In [2]:
#read data 
df_train = pd.read_csv('train.csv')
sample = np.loadtxt('sample.csv')

In [3]:
#initialize values
lamda = np.array([0.1,1,10,100,200])
X = df_train.to_numpy()[:,1:]
y = df_train.to_numpy()[:,0]
K = 10

In [20]:
#K fold cross validation loop
"""
args:
    X: complete data sets rows = features colums = samples
    y: labeles for each row of X
    K: number of Bins for K-fold cross validation
    alpha = coefficient for the ridge regression (also known als lambda)
    
return:
    errors: array of mena squared errors for of the linear fit compared to the labels
            for each fold K
"""

def cross_validation(X,y,K,alpha):

    errors = np.zeros(K)#alocate memmory for the erros we want to store
    for k in range(0,K):
        #calc interval [a,b] which we want to use as validation set
        length = int(X.shape[0]/K)
        a = k*length
        b = (k+1)*length-1
        #initialize validatio set D_k
        D_k = X[a:b,:]
        y_k = y[a:b]
        #initialize the training set which is X without D_k
        X_train = np.delete(X,slice(a,b),0)
        y_train = np.delete(y,slice(a,b))
        
        #train the liner model using ridge regression on the fold K
        reg_ridge = Ridge(alpha = alpha,fit_intercept=False, tol=1e-9,max_iter=1000000).fit(X_train, y_train)
        w_k = reg_ridge.coef_
        #print(reg_ridge.score(X_train,y_train))
        #use the validation set to compute and store the error of our fit
        y_predict = D_k @ w_k
        errors[k] = mean_squared_error(y_k, y_predict)**0.5
        #print(np.sqrt(np.mean((y_k- y_predict)**2)))
        

    return errors

In [21]:
#loops over the different lambda and computes the average error for each 
results = np.zeros_like(lamda)
for i in range(lamda.size):
    print(f'K-fold ridge cross validation for lambda = {lamda[i]}')
    errors = cross_validation(X,y,K,lamda[i])#array of errors for each bin K for one lambda
    results[i] = np.mean(errors)
    

#np.savetxt('ridge_try02.csv', results, delimiter=",")
print(results)

K-fold ridge cross validation for lambda = 0.1
K-fold ridge cross validation for lambda = 1.0
K-fold ridge cross validation for lambda = 10.0
K-fold ridge cross validation for lambda = 100.0
K-fold ridge cross validation for lambda = 200.0
[5.43676826 5.40757647 5.36103627 5.76033067 6.05113517]
